In [38]:
# from importlib import reload  # Python 3.4+

import pandas as pd
import openpyxl
from datetime import datetime
import utils
import re
import importlib
importlib.reload(utils)


regex = re.compile("[0-9]{4}\-[0-9]{2}\-[0-9]{2}")

# Note1: wait time between visit start date to pia is the most important data to look at. The goal is to reduce this time (ed wati time)




read_file = pd.read_excel ("Synthetic_ExLED_2022-01-02.xlsx", sheet_name='Synthetic ED Data for Process M')
read_file.to_csv ("Synthetic_ExLED_2022-01-02.csv", 
                  index = None,
                  header=True
                  )
df = pd.DataFrame(pd.read_csv("Synthetic_ExLED_2022-01-02.csv"))
sourceDateFormat = '%Y-%m-%d %H:%M:%S'
finalDateFormat = '%m/%d/%Y %H:%M'
identity = 1
transformedDataList = []
additionalFeatures = []
singleTrace = []
filteredEvents = ["1st lab-encounter start date", "2nd lab-encounter start date", "1st di-encounter start date", "2nd di-encounter start date", "initial disposition", "patient service", "1st lab-date-time received", "2nd lab-date-time received", "1st di-encounter end date", "1st di-encounter start date", "2nd di-encounter end date", "2nd di-encounter start date", "1st lab-encounter end date", "2nd lab-encounter end date"]
counter = 0

print(df['visit end date'][0])
print(df['time of final disposition'][0])

a = pd.to_datetime(df['visit end date'][0])
b = pd.to_datetime(df['time of final disposition'][0])


# iterate over rows
for rowIndex, row in df.iterrows():
    #iterate over each column
    for columnIndex, value in row.items():
        if pd.isna(value):
            continue
        if re.match(regex, str(value)):
            #Filtering bad events
            if columnIndex in filteredEvents:
                continue
            # Change the format of the date 
            singleTrace.append({"identity" : identity, "timestamp" : datetime.strptime(value, sourceDateFormat).strftime(finalDateFormat), "activity" : columnIndex})
            # counter = counter + 1
            continue
        additionalFeatures.append({"identity" : identity, columnIndex : value})
    # increment the identity counter
    for additionalEvent in additionalFeatures:
        for event in singleTrace:
            if additionalEvent['identity'] == event['identity']:
                event[list(additionalEvent)[1]] = additionalEvent[list(additionalEvent)[1]]
    transformedDataList = transformedDataList + singleTrace
    additionalFeatures.clear()
    singleTrace.clear()
    identity = identity + 1


# print(transformedDataList)
df_transformed = pd.DataFrame.from_dict(transformedDataList)
print("identity count: ",identity)

df_final = df_transformed.apply(lambda x: pd.Series(x.dropna().values))
# Convert dataframe to csv format
df_final.to_csv('transformedData.csv', index = False)


# read CSV file
results = pd.read_csv('transformedData.csv')
  
# count no. of lines
print("Number of lines present:-", 
      len(results))

identity count:  1050
